In [5]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
import numpy as np

In [6]:
def readArticle(input_file_path):
    file = open(input_file_path, 'r', encoding = 'utf8')
    articles_dt = file.read().split('\n')[:-1]
    pubId, canonicalUrl,firstScrape,title,text,lang_reliability = [],[],[],[],[],[]
    for article in articles_dt:    
        row = article.split('\t')
        pubId.append(row[0])
        canonicalUrl.append(row[4])
        firstScrape.append(row[5])
        lang_reliability.append(row[7])
        title.append(row[8])
        text.append(row[9])
    articles_df = pd.DataFrame()
    articles_df['pubId'], articles_df['canonicalUrl'], articles_df['firstScrape'], articles_df['title'], articles_df['text'], articles_df['lang_reliability']= pubId, canonicalUrl,firstScrape,title,text,lang_reliability
    return articles_df

In [7]:
## text normzalization
STOPWORDS = set(stopwords.words('english'))
def clean_text(text):
    """
        text: a paragrapy
        
        return: modified initial string
    """
    text = re.sub(r"[^\w\s]", '', text) ## Remove all non-word characters (everything except numbers and letters)
    text = text.lower() # lowercase text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwords from text
    return text

In [8]:
def load_GloVe():
    ## word embedding -GloVe
# load GloVe 
    embeddings_index = {}
    f = open('../data/embedding/glove.6B/glove.6B.100d.txt')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    return embeddings_index

In [9]:
def count_Len_Sum(embeddings_index, news_title):
    header_length_dict ={} #key: header_index, value: length of header 
    header_sum_dict = {} #key: index, value: the matrix of the header 
    for i in news_title.index:
        words_list = news_title[i].split(' ')
        #The dimension of the matrix is 10 0 because the glove embedding vector sizes I used is 100.
        header_sum = [0]*100
        for word in words_list:
            if word in embeddings_index:
                word_embedding_vector = embeddings_index.get(word)
                header_sum += word_embedding_vector
        header_length_dict[i] = len(words_list)  
        header_sum_dict[i] = header_sum 
    return header_length_dict, header_sum_dict

In [10]:
def find_same_article(news_title,header_length_dict, header_sum_dict ):
    # find similary article for article 1
    similiary_dict = {}
    for i in news_title.index:
        target_length = header_length_dict[i]
        target_hearder_vec = header_sum_dict[i]
        find_keys = []
        for key in header_length_dict.keys():
            if header_length_dict[key] == target_length:
                non_vec = [0] *100
                try:
                    if sum(header_sum_dict[key] - target_hearder_vec) == 0:
                        find_keys.append(key)
                except:
                    #print('type header_sum_dict', type(header_sum_dict[key]))
                    #print(header_sum_dict[key])
                    print(key)
                    #print(type(target_hearder_vec))    
        similiary_dict[i] = find_keys
    return similiary_dict
        

In [11]:
if __name__ == '__main__':
    df = readArticle('../data/raw/articles.txt')
    print(df.shape)
    title_df = df['title'].apply(clean_text)
    embeddings_index = load_GloVe()
    print(embeddings_index['cat'])
    #use full articles to train the dictionaies
    header_length_dict, header_sum_dict = count_Len_Sum(embeddings_index, title_df)
    #exam the top 50 ones to find the similar titles
    sim_dict = find_same_article(title_df[:50],header_length_dict, header_sum_dict )
    

(213605, 6)
[ 0.23088    0.28283    0.6318    -0.59411   -0.58599    0.63255
  0.24402   -0.14108    0.060815  -0.7898    -0.29102    0.14287
  0.72274    0.20428    0.1407     0.98757    0.52533    0.097456
  0.8822     0.51221    0.40204    0.21169   -0.013109  -0.71616
  0.55387    1.1452    -0.88044   -0.50216   -0.22814    0.023885
  0.1072     0.083739   0.55015    0.58479    0.75816    0.45706
 -0.28001    0.25225    0.68965   -0.60972    0.19578    0.044209
 -0.31136   -0.68826   -0.22721    0.46185   -0.77162    0.10208
  0.55636    0.067417  -0.57207    0.23735    0.4717     0.82765
 -0.29263   -1.3422    -0.099277   0.28139    0.41604    0.10583
  0.62203    0.89496   -0.23446    0.51349    0.99379    1.1846
 -0.16364    0.20653    0.73854    0.24059   -0.96473    0.13481
 -0.0072484  0.33016   -0.12365    0.27191   -0.40951    0.021909
 -0.6069     0.40755    0.19566   -0.41802    0.18636   -0.032652
 -0.78571   -0.13847    0.044007  -0.084423   0.04911    0.24104
  0.45273

In [12]:
sim_dict

{0: [0],
 1: [1],
 2: [2, 108875, 108985, 123922, 135832, 140029, 168704, 200409],
 3: [3],
 4: [4],
 5: [5],
 6: [6],
 7: [7, 123918],
 8: [8],
 9: [9],
 10: [10],
 11: [11],
 12: [12],
 13: [13, 200731],
 14: [14],
 15: [15, 200657],
 16: [16],
 17: [17],
 18: [18, 73615],
 19: [19, 200530],
 20: [20],
 21: [21],
 22: [22],
 23: [23],
 24: [24],
 25: [25],
 26: [26, 200741],
 27: [27],
 28: [28],
 29: [29, 200856],
 30: [30],
 31: [31],
 32: [32],
 33: [33],
 34: [34],
 35: [35],
 36: [36],
 37: [37],
 38: [38],
 39: [39],
 40: [40],
 41: [41],
 42: [42],
 43: [43],
 44: [44],
 45: [45],
 46: [46],
 47: [47],
 48: [48],
 49: [49]}

In [24]:
print(df['title'][2])
print(df['title'][108875])
print(df['title'][108985])
print(df['title'][123922])

Gallup Shows How Much Americans Really Care About The "Situation With Russia
GALLUP SHOWS HOW MUCH AMERICANS REALLY CARE ABOUT THE "SITUATION WITH RUSSIA
GALLUP SHOWS HOW MUCH AMERICANS REALLY CARE ABOUT THE "SITUATION WITH RUSSIA
Gallup Shows How Much Americans Really Care About The "Situation With Russia"


In [20]:
print(df['text'][2])

by Knave Dave - Jul 18, 2018 1:11 pm ### This is EXACTLY what the first stage of a housing collapse feels like. ### Jul 18, 2018 1:11 PM ### by quoth the raven - Jul 18, 2018 6:12 pm ### Bill explains his reasoning and why he believes Paul Krugman is a "world class idiot". ### Jul 18, 2018 6:12 PM ### 0 ### SHARES ### While ever-hope-filled expectations among the left are for a 'blue wave' in the Mid-term elections, we suspect things may not turn out quite as planned given the last week's "crisis". Even before President Trump had set foot in Helsinki, the left and the media were banging the drums of war against "the thug" Putin and how he would trump Trump, and once the press conference furore was over, all hell broke loose as the left-leaning world attempted to out-signal one another's virtue as to the "treasonous", "surrender" that had occurred. ### So much so - in fact - that the internet became 'full' of "Trump" and "Treason" chatter - more so even than when he joked in July 2016 a

In [21]:
print(df['text'][108875])

GALLUP SHOWS HOW MUCH AMERICANS REALLY CARE ABOUT THE "SITUATION WITH RUSSIA" ### While ever-hope-filled expectations among the left are for a 'blue wave' in the Mid-term elections, we suspect things may not turn out quite as planned given the last week's "crisis". Even before President Trump had set foot in Helsinki, the left and the media were banging the drums of war against "the thug" Putin and how he would trump Trump, and once the press conference furore was over, all hell broke loose as the left-leaning world attempted to out-signal one another's virtue as to the "treasonous", "surrender" that had occurred. ### So much so - in fact - that the internet became 'full' of "Trump" and "Treason" chatter - more so even than when he joked in July 2016 about Russia having Hillary's missing 30,000 emails. ### Once again - the "Russia, Russia, Russia" cries drowned out any sane discussion of the end of cold war 2.0. ### There's just one problem with the Democrats' unending focus on Russia 

In [23]:
print(df['text'][108985])

GALLUP SHOWS HOW MUCH AMERICANS REALLY CARE ABOUT THE "SITUATION WITH RUSSIA" ### While ever-hope-filled expectations among the left are for a 'blue wave' in the Mid-term elections, we suspect things may not turn out quite as planned given the last week's "crisis". Even before President Trump had set foot in Helsinki, the left and the media were banging the drums of war against "the thug" Putin and how he would trump Trump, and once the press conference furore was over, all hell broke loose as the left-leaning world attempted to out-signal one another's virtue as to the "treasonous", "surrender" that had occurred. ### So much so - in fact - that the internet became 'full' of "Trump" and "Treason" chatter - more so even than when he joked in July 2016 about Russia having Hillary's missing 30,000 emails. ### Once again - the "Russia, Russia, Russia" cries drowned out any sane discussion of the end of cold war 2.0. ### There's just one problem with the Democrats' unending focus on Russia 

In [25]:
print(df['text'][123922])

Source: Gallup ### As David Sirota  noted , “Gallup recently did a poll of what Americans say is the most important problem facing the country. One finding: the percentage of Americans saying “Situation with Russia” is the most important problem is literally too small to represent with a number. “ ### So with all that energy expended on pinning the Russia collusion tail on Trump’s campaign donkey, let alone former FBI Director Comey’s insistence that ‘anyone voting Republican is anti-American’, we are reminded of the blinkered view of the world so many suffer from and what Steve Bannon said yesterday at  CNBC’s “Delivering Alpha” conference : The Democrats abandoned the American worker. ### The meddling was on the margin, and the Democrats need to accept this. The collusion – they haven’t found one shred of evidence. ### The Democrats have yet to embrace why they lost…[they] have wanted a do-over since 2:30 am on Nov. 9 – and they’ve lied and they’ve bitched and in November they’ll get

In [26]:
print(df['title'][13])
print(df['title'][200731])


If Everything's So Awesome, Why Are Investors Paying So Much For Crash Protection
If Everything’s So Awesome, Why Are Investors Paying So Much For Crash Protection


In [27]:
print(df['text'][13])

by Phoenix Capita… - Jul 19, 2018 8:49 am ### The Government is planning a $1 trillion deficit next year. ### Jul 19, 2018 8:49 AM ### Financial Terrorism In The UK – Collusion between Government, Regulators & Two Bailed-Out UK Banks ### by GoldCore - Jul 19, 2018 8:01 am ### Neil Mitchell has waged a long battle for justice for UK victims of the UK taxpayer owned bank RBS and HBOS ### Jul 19, 2018 8:01 AM ### 0 ### SHARES ### Tech stocks soaring to record highs and a slump in volatility suggest investors are brushing aside market risks, trade war concerns, and Central Bank tightening. ### However, as Bloomberg notes, there is at least one gauge of investor sentiment that hints at a growing concern. The Cboe Skew Index, which tracks the cost of tail-risk equity protection, has jumped to the highest level since October. The rise signals options traders are growing wary of wild swings, just as the International Monetary Fund warned financial markets seem complacent to mounting risks in t

In [28]:
print(df['text'][200731])

Gold and Precious Metals ### If Everything’s So Awesome, Why Are Investors Paying So Much For Crash Protection? ### Thursday, July 19, 2018 13:26 ### % of readers think this story is Fact. Add your two cents. ### Tech stocks soaring to record highs and a slump in volatility suggest investors are brushing aside market risks, trade war concerns, and Central Bank tightening. ### However, as Bloomberg notes, there is at least one gauge of investor sentiment that hints at a growing concern. The Cboe Skew Index, which tracks the cost of tail-risk equity protection, has jumped to the highest level since October. The rise signals options traders are growing wary of wild swings, just as the International Monetary Fund warned financial markets seem complacent to mounting risks in the global economy. ### Generally, a rise in skew indicates that ‘crash protection’ is in demand among institutional investors (institutional/professional investors are the biggest traders in SPX options). ### But an un